In [1]:
from datasets import load_dataset

from llm import chat_completion, yesno_completion

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Replace with desired dataset (from HuggingFace)
hf_dataset_name = "hendrycks/competition_math"
dataset = load_dataset(hf_dataset_name)

# Example item from "hendrycks/competition_math"
"""
{'problem': 'A board game spinner is divided into three parts labeled $A$, $B$  and $C$. The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is the probability of the spinner landing on $C$? Express your answer as a common fraction.',
 'level': 'Level 1',
 'type': 'Counting & Probability',
 'solution': 'The spinner is guaranteed to land on exactly one of the three regions, so we know that the sum of the probabilities of it landing in each region will be 1. If we let the probability of it landing in region $C$ be $x$, we then have the equation $1 = \\frac{5}{12}+\\frac{1}{3}+x$, from which we have $x=\\boxed{\\frac{1}{4}}$.'}

"""

In [ ]:
# Ask for GPT's completion to 'problem' column

# Compare to dataset 'solution' column

# From Experiment1

When the ai considers whether to step back or not, does its one-word answer differ from its CoT answer?

In [1]:
from dataclasses import dataclass
import sample

@dataclass
class Result:
    context: sample.Context
    one_token_cpc_result: str
    cot_cpc_result: str

from dotenv import load_dotenv
load_dotenv()

from llm import LLM
from openai import OpenAI
llm = LLM(OpenAI(), "gpt-3.5-turbo")

Each 'passage' is a lengthy text where we are reasoning through a problem.
Consider progressively larger context parts of each passage (that is, we are 'checking in' as we proceed through reading the passage):

In [2]:
import json
passages = json.load(open("data/passages1.json"))
checkpoints = (text for document in passages for text in sample.checkpoints(document, 1000))

and for each context part, ask the llm if the current approach is working or not.

In [3]:
from solver import perform_one_token_cpc, perform_cot_cpc

results = [
    Result(
        context=context,
        one_token_cpc_result=perform_one_token_cpc(llm, context),
        cot_cpc_result=perform_cot_cpc(llm, context)
    )
    for context in checkpoints
]

For each result, determine whether the result is good (the two cpc methods agreed) or bad (they disagreed).

In [8]:
from solver import perform_one_token_cpc, perform_cot_cpc
from judge import JudgeResult

evaluations = [
    JudgeResult(
        result=result,
        score=1.0 if result.one_token_cpc_result.lower() == result.cot_cpc_result.lower() else 0.0
    )
    for result in results
]

In [10]:
for evaluation in evaluations[:10]:
    print(f"For context ending in '...{evaluation.result.context.text[-90:]}'")
    print(f"one_token_cpc_result={evaluation.result.one_token_cpc_result}, "
          f"cot_cpc_result={evaluation.result.cot_cpc_result}, score={evaluation.score}")

For context ending in '...sonal and individualized concept that each person must discover and define for themselves.'
one_token_cpc_result=Adapt, cot_cpc_result=, score=0.0
